**Connect google drive**

In [15]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'My Drive'   Untitled   Untitled0.ipynb   yolov3


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14358, done.
remote: Total 14358 (delta 0), reused 0 (delta 0), pack-reused 14358
Receiving objects: 100% (14358/14358), 13.08 MiB | 20.60 MiB/s, done.
Resolving deltas: 100% (9764/9764), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=17@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=17@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=17@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=66@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=66@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=66@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [8]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-11-03 17:31:02--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   179KB/s    in 16m 53s 

2020-11-03 17:47:57 (157 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/img_obj_det1_185.txt  
  inflating: data/obj/img_obj_det1_291.jpg  
  inflating: data/obj/img_obj_det1_291.txt  
  inflating: data/obj/img_obj_det1_406.jpg  
  inflating: data/obj/img_obj_det1_406.txt  
  inflating: data/obj/img_obj_det1_581.jpg  
  inflating: data/obj/img_obj_det1_581.txt  
  inflating: data/obj/classes.txt    
  inflating: data/obj/img_obj_det1_7.jpg  
  inflating: data/obj/img_obj_det1_7.txt  
  inflating: data/obj/img_obj_det1_21.jpg  
  inflating: data/obj/img_obj_det1_21.txt  
  inflating: data/obj/img_obj_det1_117.jpg  
  inflating: data/obj/img_obj_det1_117.txt  
  inflating: data/obj/img_obj_det1_120.jpg  
  inflating: data/obj/img_obj_det1_120.txt  
  inflating: data/obj/img_obj_det1_185.jpg  


In [11]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

1 0.338802 0.500000 0.008854 0.035185

['1', '0.338802', '0.500000', '0.008854', '0.035185']
0 data/obj/img_obj_det1_406.txt
1 0.338802 0.500000 0.008854 0.035185
1 0.585938 0.577778 0.012500 0.025926

['1', '0.585938', '0.577778', '0.012500', '0.025926']
0 data/obj/img_obj_det1_406.txt
1 0.585938 0.577778 0.012500 0.025926
1 0.467708 0.453241 0.010417 0.037963

['1', '0.467708', '0.453241', '0.010417', '0.037963']
0 data/obj/img_obj_det1_406.txt
1 0.467708 0.453241 0.010417 0.037963
1 0.610156 0.331944 0.011979 0.028704

['1', '0.610156', '0.331944', '0.011979', '0.028704']
0 data/obj/img_obj_det1_406.txt
1 0.610156 0.331944 0.011979 0.028704
1 0.736198 0.477778 0.015104 0.040741

['1', '0.736198', '0.477778', '0.015104', '0.040741']
0 data/obj/img_obj_det1_406.txt
1 0.736198 0.477778 0.015104 0.040741
1 0.648958 0.550000 0.013542 0.040741

['1', '0.648958', '0.550000', '0.013542', '0.040741']
0 data/obj/img_obj_det1_406.txt
1 0.648958 0.550000 0.013542 0.040741
16 0.180729 0.793056 0

In [12]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/img_obj_det1_185.jpg', 'data/obj/img_obj_det1_581.jpg', 'data/obj/img_obj_det1_406.jpg', 'data/obj/img_obj_det1_7.jpg', 'data/obj/img_obj_det1_21.jpg', 'data/obj/img_obj_det1_291.jpg', 'data/obj/img_obj_det1_117.jpg', 'data/obj/img_obj_det1_120.jpg']


In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
print ("done")